<a href="https://colab.research.google.com/github/FinancialRADDeveloper/GoogleMusicDataAnalysis/blob/master/GoogleMusicAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# !pip install spotipy

import pandas as pd

df = pd.DataFrame([1,2,2,222,3,5], columns=['numbers'])

df.numbers.tolist()

# import spotipy


[1, 2, 2, 222, 3, 5]

In [20]:
x = input('Client Code: ')

print(x * 2)

Client Code: dwecv  vghv thdfv ythdv thvfth vh 
dwecv  vghv thdfv ythdv thvfth vh dwecv  vghv thdfv ythdv thvfth vh 
